In [56]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

In [93]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100, frac=0.2):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}-{frac}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp]} for sp in scores})
    df.loc[[u[:5]=="usock" for u in df.index.tolist()], "truth"] = 1
    df.loc[[u[:6]=="udummy" for u in df.index.tolist()], "truth"] = 1
    return df

In [105]:
def get_metrics(df_res, splits=4, K=None, th=0.5):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        # df_res["truth"] = df_res["truth"].fillna(1)
#         df_res.loc[[u[:5]=="usock" for u in df_res.index.tolist()], "truth"] = 1
#         df_res.loc[[u[:6]=="udummy" for u in df_res.index.tolist()], "truth"] = 0
        df = df_res[[sp, "truth"]].dropna()
        t = df["truth"].sum() if K is None else K
        t_frac = t / df.shape[0]
        # df["pred"] = (df[sp].rank(ascending=False, method="min") <= t) * 1
        # df["pred"] = df[sp] >= t_frac
        df["pred"] = df[sp] >= th

        ret[sp] = {
            "precision": sklearn.metrics.precision_score(y_true=df["truth"], y_pred=df["pred"]),
            "recall": sklearn.metrics.recall_score(y_true=df["truth"], y_pred=df["pred"]),
            "f1": sklearn.metrics.f1_score(y_true=df["truth"], y_pred=df["pred"])
        }
    return ret

In [106]:
alg_list = ["fraudar", "rsd", "rev2"][:2]
data_list = ["alpha", "otc", "amazon", "epinions"][:1]
budget_list = [100.0, 200.0, 300.0, 400.0]
frac_list = [0.2, 0.4, 0.6, 0.8]

In [118]:
attack_res = {
    (d, a, b, f): get_results(data_name=d, alg=a, attack_name="ilp_attack", budget=b, frac=f)
    for d in data_list for a in alg_list
    for b in budget_list for f in frac_list
}
dict_met = {
    (d, a, b, f): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b, f)], splits=4, K=None, th=1.2 if a == "rsd" else 0.5))
    for d, a, b, f in attack_res
#     for b in budget_list for f in frac_list
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
display(df_met)

0         1         2         3
alpha fraudar 100.0 0.2 precision  0.350785  0.370968  0.380682  0.335526
                        recall     0.761364  0.741935  0.770115  0.629630
                        f1         0.480287  0.494624  0.509506  0.437768
                    0.4 precision  0.364103  0.364641  0.380682  0.364198
                        recall     0.710000  0.628571  0.676768  0.634409
...                                     ...       ...       ...       ...
      rsd     400.0 0.6 recall     0.186364  0.195556  0.146119  0.183099
                        f1         0.310606  0.323529  0.252964  0.307087
                    0.8 precision  0.943396  0.938776  0.952381  0.965517
                        recall     0.186567  0.168498  0.149813  0.214559
                        f1         0.311526  0.285714  0.258900  0.351097

[96 rows x 4 columns]

In [125]:
# attack_res[("alpha", "rsd", 400.0, 0.2)].to_csv("ar2.csv")

In [119]:
pd.concat([df_met.loc[("alpha", "fraudar", slice(None), slice(None))], df_met.loc[("alpha", "fraudar", slice(None), slice(None))].mean(axis=1)], axis=1)

0         1         2         3         0
100.0 0.2 precision  0.350785  0.370968  0.380682  0.335526  0.359490
          recall     0.761364  0.741935  0.770115  0.629630  0.725761
          f1         0.480287  0.494624  0.509506  0.437768  0.480546
      0.4 precision  0.364103  0.364641  0.380682  0.364198  0.368406
          recall     0.710000  0.628571  0.676768  0.634409  0.662437
          f1         0.481356  0.461538  0.487273  0.462745  0.473228
      0.6 precision  0.364103  0.370968  0.380682  0.364198  0.369987
          recall     0.633929  0.589744  0.603604  0.561905  0.597295
          f1         0.462541  0.455446  0.466899  0.441948  0.456708
      0.8 precision  0.357513  0.370968  0.380682  0.364198  0.368340
          recall     0.556452  0.534884  0.544715  0.504274  0.535081
          f1         0.435331  0.438095  0.448161  0.422939  0.436132
200.0 0.2 precision  0.357513  0.377660  0.380682  0.364198  0.370013
          recall     0.690000  0.676190  0.676768  0.634409  0.669342
          f1         0.470990  0.484642  0.487273  0.462745  0.476412
      0.4 precision  0.357513  0.382199  0.380682  0.335526  0.363980
          recall     0.556452  0.565891  0.544715  0.435897  0.525739
          f1         0.435331  0.456250  0.448161  0.379182  0.429731
      0.6 precision  0.357513  0.377660  0.380682  0.335526  0.362845
          recall     0.466216  0.464052  0.455782  0.361702  0.436938
          f1         0.404692  0.416422  0.414861  0.348123  0.396024
      0.8 precision  0.357513  0.380208  0.380682  0.364198  0.370650
          recall     0.401163  0.412429  0.391813  0.357576  0.390745
          f1         0.378082  0.395664  0.386167  0.360856  0.380192
300.0 0.2 precision  0.357513  0.398990  0.384181  0.364198  0.376220
          recall     0.616071  0.675214  0.612613  0.561905  0.616451
          f1         0.452459  0.501587  0.472222  0.441948  0.467054
      0.4 precision  0.357513  0.386598  0.404372  0.364198  0.378170
          recall     0.466216  0.490196  0.503401  0.418440  0.469563
          f1         0.404692  0.432277  0.448485  0.389439  0.418723
      0.6 precision  0.357513  0.389744  0.473430  0.364198  0.396221
          recall     0.375000  0.402116  0.535519  0.333333  0.411492
          f1         0.366048  0.395833  0.502564  0.348083  0.403132
      0.8 precision  0.364103  0.511905  0.380682  0.364198  0.405222
          recall     0.322727  0.573333  0.305936  0.276995  0.369748
          f1         0.342169  0.540881  0.339241  0.314667  0.384239
400.0 0.2 precision  0.357513  0.440909  0.380682  0.335526  0.378658
          recall     0.556452  0.751938  0.544715  0.435897  0.572251
          f1         0.435331  0.555874  0.448161  0.379182  0.454637
      0.4 precision  0.357513  0.383420  0.380682  0.356250  0.369466
          recall     0.401163  0.418079  0.391813  0.345455  0.389127
          f1         0.378082  0.400000  0.386167  0.350769  0.378755
      0.6 precision  0.357513  0.380208  0.384181  0.364198  0.371525
          recall     0.313636  0.324444  0.310502  0.276995  0.306395
          f1         0.334140  0.350120  0.343434  0.314667  0.335590
      0.8 precision  0.357513  0.355556  0.384181  0.356250  0.363375
          recall     0.257463  0.234432  0.254682  0.218391  0.241242
          f1         0.299349  0.282561  0.306306  0.270784  0.289750

In [132]:
df_met.index = df_met.index.set_names(["data", "algo", "budget", "frac", "metrics"])
df_met.mean(axis=1).to_frame().reset_index().pivot(index=["data", "algo", "frac", "metrics"], columns=["budget"]).loc[("alpha", "rsd", slice(None), "f1")].droplevel(0, axis=1)

budget,100.0,200.0,300.0,400.0
frac,,,,
0.2,0.373942,0.358328,0.348181,0.336807
0.4,0.349987,0.334689,0.331038,0.310910
0.6,0.347477,0.326286,0.304469,0.298547
0.8,0.339316,0.328980,0.304640,0.301809
